# 웹 크롤링2 - Dynamic Crawling

## 0. 라이브러리

In [37]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import pandas as pd
from pandas import DataFrame
import time

### 1.1 Simple Text Crawling
멜론 사이트에서 노래 제목을 크롤링해보자

URL: https://www.melon.com/chart/index.htm

In [38]:
DRIVER_PATH = 'C:/Users/Chrome/chromedriver.exe'   # 드라이버 불러온다

In [39]:
# chrome driver 설정
driver = webdriver.Chrome(DRIVER_PATH)
driver.implicitly_wait(10)   # 드라이버를 여는 것을 10초까지 기다린다.

url = "https://www.melon.com/chart/index.htm"

driver.get(url)  # 크롬 드라이버에서 멜론 주소를 받는다 
html = driver.page_source   # 렌더링 된 페이지의 모든 Elements를 불러온다
soup = BeautifulSoup(html,'html.parser') # html을 객체로 저장

#title crawling
title = WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.CSS_SELECTOR, "#frm > div > table > tbody > tr:nth-child(1) > td:nth-child(4) > div > div")))
# 드라이버를 통해 여는 초를 20초로 정한다. 지나면 타임아웃 발생
# 뒤에 By.CSS_SELECTOR를 보아 ID가 아닌 셀렉터로 검색

# print("Title:{}".format(title.text))

title.text

'Dynamite\n방탄소년단'

In [40]:
# 2번째 제목 크롤링
WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.XPATH, "//*[@id='frm']/div/table/tbody/tr[2]/td[4]/div/div"))).text

# frm이란 ID를 갖는 테이블에서 그 후에 나오는 태그에 맞는 속성을 추출하여 text로 저장한다.

'취기를 빌려 (취향저격 그녀 X 산들)\n산들'

### 1.2 Text Crawling with for loop
위에서 찾은 Xpath의 규칙을 바탕으로 for loop 만들자

In [42]:
# chrome driver 설정
driver = webdriver.Chrome(DRIVER_PATH) 
driver.implicitly_wait(10)  # 대기시간 설정

url = "https://www.melon.com/chart/index.htm"     # 멜론 주소 url에 입력

driver.get(url)           # url주소 드라이버로 받음
html = driver.page_source     # 렌더링 된 페이지의 모든 Elements를 불러온다
soup = BeautifulSoup(html, 'html.parser')    # 객체로 저장

# 빈 리스트 변수
title_list = []

# title crawling (TOP 50)
for i in range(1, 51):
    title = WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.XPATH, f"//*[@id='frm']/div/table/tbody/tr[{i}]/td[4]/div/div")))
    title_list.append(title.text)
# 제목들을 추출하여 리스트에 저장

print(title_list)

chartrank = pd.DataFrame({"song&singers":title_list})  # 추후에 리스트로 나누어서 제목, 가수 등 만들어서 데이터 프레임으로 가능
chartrank.head()

['Dynamite\n방탄소년단', '취기를 빌려 (취향저격 그녀 X 산들)\n산들', '눈누난나 (NUNU NANA)\n제시 (Jessi)', '마리아 (Maria)\n화사 (Hwa Sa)', '다시 여기 바닷가\n싹쓰리 (유두래곤, 린다G, 비룡)', 'When We Disco (Duet with 선미)\n박진영', '오래된 노래\n스탠딩 에그', 'How You Like That\nBLACKPINK', '에잇(Prod.&Feat. SUGA of BTS)\n아이유', '내 마음이 움찔했던 순간 (취향저격 그녀 X 규현)\n규현 (KYUHYUN)', '아로하\n조정석', 'Dolphin\n오마이걸 (OH MY GIRL)', 'Downtown Baby\n블루 (BLOO)', '어떻게 지내 (Prod. By VAN.C)\n오반', '홀로\n이하이', 'Summer Hate (Feat. 비)\n지코 (ZICO)', 'Blueming\n아이유', 'Dance Monkey\nTones And I', '덤디덤디 (DUMDi DUMDi)\n(여자)아이들', '흔들리는 꽃들 속에서 네 샴푸향이 느껴진거야\n장범준', 'Memories\nMaroon 5', '살짝 설렜어 (Nonstop)\n오마이걸 (OH MY GIRL)', 'METEOR\n창모 (CHANGMO)', 'Not Shy\nITZY (있지)', '어떻게 이별까지 사랑하겠어, 널 사랑하는 거지\nAKMU (악동뮤지션)', '작은 것들을 위한 시 (Boy With Luv) (Feat. Halsey)\n방탄소년단', '사랑은 지날수록 더욱 선명하게 남아\n전상근', '사랑하게 될 줄 알았어\n전미도', '늦은 밤 너의 집 앞 골목길에서\n노을', 'Ice Cream (with Selena Gomez)\nBLACKPINK', '그 여름을 틀어줘\n싹쓰리 (유두래곤, 린다G, 비룡)', '거짓말이라도 해서 널 보고싶어\n백지영', '보라빛 밤 (pporappippam)\n선미', '모든 날, 모든 순간 (Every day

,song&singers
0,Dynamite\n방탄소년단
1,취기를 빌려 (취향저격 그녀 X 산들)\n산들
2,눈누난나 (NUNU NANA)\n제시 (Jessi)
3,마리아 (Maria)\n화사 (Hwa Sa)
4,"다시 여기 바닷가\n싹쓰리 (유두래곤, 린다G, 비룡)"


### 1.3 Text Crawling (Click & Back)
클릭하고 나오기 -> 동적 크롤링 가능 (가사 크롤링 가능)


In [43]:
# chrome driver 설정
driver = webdriver.Chrome(DRIVER_PATH)
driver.implicitly_wait(10)

url = "https://www.melon.com/chart/index.htm"  # url 주소 입력
driver.get(url)
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

# 1번째 클릭하기
click_element = WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.XPATH, '//*[@id="frm"]/div/table/tbody/tr[1]/td[3]/div/a')))
click_element.click()

# back
driver.back()

# 2번째 클릭하기
click_element = WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.XPATH, '//*[@id="frm"]/div/table/tbody/tr[2]/td[3]/div/a')))
click_element.click()

# back
driver.back()

### 1.4 Text Crawling including contents
* 1.2처럼 for문과 함께 써보자! (첫 페이지 5개의 글에 대해 title, artist, heart(하트갯수), lyrics(가사)를 크롤링
* 1.3에서 사용한 click & back을 활용하자

In [44]:
# chrome driver 설정
driver = webdriver.Chrome(DRIVER_PATH)
driver.implicitly_wait(10)

url = "https://www.melon.com/chart/index.htm"
driver.get(url)
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

# 빈 리스트 변수
title_list = []
artist_list = []
heart_list = []
lyrics_list = []

# crawling (TOP 5)
for i in range(1, 6):
    # click
    click_element = WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.XPATH, f'//*[@id="frm"]/div/table/tbody/tr[{i}]/td[3]/div/a')))
    # 페이지 별 요소들
    click_element.click()   # 요소 클릭

    # title crawling
    title = WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.CSS_SELECTOR, "#downloadfrm > div > div > div.entry > div.info > div.song_name")))
    title_list.append(title.text)
    # 타이틀 명 리스트에 추가
    
    # artist crawling
    artist = WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.CSS_SELECTOR, "#downloadfrm > div > div > div.entry > div.info > div.artist > a > span:nth-child(1)")))
    artist_list.append(artist.text)
    # 아티스트 명 리스트에 추가
    
      # heart crawling
    heart = WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.CSS_SELECTOR, "#d_like_count")))
    heart_list.append(heart.text)
    # 하트 갯수 리스트에 추가
    
    # lyrics crawling
    lyrics = WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.CSS_SELECTOR, "#d_video_summary")))
    lyrics_list.append(lyrics.text)
    # 가사 리스트에 추가
    
# back
    driver.back()  # 나오기
    
print(title_list)
print(artist_list)
print(heart_list)
print(lyrics_list)

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=85.0.4183.121)


### TIP: 보통은 결과값을 데이터프레임 형태로 저장한다

In [ ]:
# 결과 변수
# raw_result에 각 제목, 아티스트, 하트 개수 등을 저장
raw_result = {'title':title_list, 'artist':artist_list, 'heart':heart_list, 'lyrics':lyrics_list}
result = pd.DataFrame(raw_result)  # CSV 파일로 저장

# driver 종료
driver.quit()
result

## 2. Image Crawling
이미지 크롤링하기

#### STEP1. URL Crawling

In [ ]:
# crhome driver설정
driver = webdriver.Chrome(DRIVER_PATH)
driver.implicitly_wait(10)

url = "https://www.melon.com/chart/index.htm"

driver.get(url)
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

# 빈 리스트 변수
link_list = []

# 이미지 크롤링 (TOP 50)
for i in range(1,51):
    img = WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.CSS_SELECTOR, f"#frm > div > table > tbody > tr:nth-child({i}) > td:nth-child(2) > div > a > img")))
    link_list.append(img.get_attribute('src')) # 지속해서 link_list에 img 소스 추가해줌

print(link_list)

#### STEP2. Download images using URLs
자신의 디렉토리에 img 폴더 생성하고 실행

In [ ]:
import urllib.request

count = 0
for link in link_list:
    count += 1
    urllib.request.urlretrieve(link, './img/img' + str(count) + '.jpg.')
# link_list의 요소 갯수들을 다 파일로 내보냄